<a href="https://colab.research.google.com/github/vijayyedidi/Reading_Google_Sheets/blob/main/connecting_google_drive_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Main data processing packages
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', '{:.2f}'.format)
import numpy as np

from tqdm import tqdm
import sys
import os


## Visualization
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

## Google Imports
from google.colab import auth
from google.cloud import bigquery
import gspread
from google.auth import default
import gspread_dataframe as gd
try:
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)
  print('authenticated')

except:
  print('authentication failed')
  sys.exit(1)

class ReadDrive: ## Class to connect to google sheets and read/write data
  ## Read Google Sheets Link to be able to read to or write sheets

  def __init__(self, sheet_url, bq = False):

    self.gc = gspread.authorize(creds) ## Open the function to be able to read Google Sheets
    self.sheet_url = sheet_url
    self.wb = self.gc.open_by_url(self.sheet_url)

  def ReadSheet(self, sheet_name):
    ## Given the sheet name read in the sheet data
    self.df = pd.DataFrame(self.wb.worksheet(sheet_name).get_all_values()) ## Save as DataFrame
    self.df = pd.DataFrame(self.df.values[1:],columns = self.df.iloc[0]) ## Make the first Column the header, values below
    self.df = self.determine_data_type(self.df)

    return self.df

  def WriteSheet(self, sheet_name, df):
    ## Given the sheet name on a spreadsheet write data to this sheet
    try:
      sheet = self.wb.worksheet(sheet_name)
    except:
      sheet = self.wb.add_worksheet(title = sheet_name, rows = '100', cols = '20')
    gd.set_with_dataframe(sheet, df)

  def determine_data_type(self, df):
    for col in df.columns:
      df[col] = df[col].fillna('').astype(str).str.replace(',', '', regex = True).str.replace('$', '', regex = True)
      if df[col].str.contains(r'[^0-9\.]').any() or df[col].str.contains(r'[A-Za-z]').any():
        df[col] = df[col].astype(str)
      else:
        df[col] = pd.to_numeric(df[col], errors = 'coerce')
        df[col] = df[col].astype(float)
    return df.replace('', np.nan)

class Read_Big_Query(): ## Class to connect to bigquery and pull into dataframes
  def __init__(self):
    self.project = 'essence-analytics-dwh'
    self.location = 'US'
    self.client = bigquery.Client(project = self.project, location = self.location)
    try:
      auth.authenticate_user()
      print("Authenticated")
    except:
      print('Unable to Authenticate User')
      sys.exit(1)
  def bq_pull(self, query):
    query_job = self.client.query(query)
    query_job = query_job.to_dataframe()
    query_job = query_job.replace("<NA>", np.nan)
    return query_job